# Setting Environment

In [1]:
# !pip install datasets
# !pip install evaluate
# !pip install accelerate -U
# !pip install transformers[torch]
# !pip install rouge
# !pip install rouge_score

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import torch
from torch import nn
from datasets import Dataset
from transformers import BertModel, BertTokenizer, BertConfig, BertForSequenceClassification
from transformers import EncoderDecoderConfig, EncoderDecoderModel
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
from rouge import Rouge
from datasets import load_metric
from transformers import PretrainedConfig
from transformers import BertConfig, EncoderDecoderConfig
from transformers import BertPreTrainedModel, BertModel
from transformers.models.bart.modeling_bart import BartForConditionalGeneration
import transformers
from transformers.generation_utils import GenerationMixin

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [4]:
transformers.__version__

'4.35.2'

# Extract The Data

In [5]:
# # exctract from specific folder in tar file
# # tar -zxvf [tar file path] [directory target extracted data] [directory target on tar file]

# # EXTRME DATA
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/xtrme/
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/xtrme/test/
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/xtrme/dev/

# # CANONICAL DATA
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/canonical/
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/canonical/train/
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/canonical/test/
# !tar -zxvf /content/drive/MyDrive/liputan6_data.tar.gz C /content/drive/MyDrive/SUMMARIZATION_MODEL_FINE_TUNING/id_liputan6/ liputan6_data/canonical/dev/

## Extract JSON Data

## Convert Data to CSV

## Get Datasets

In [6]:
train_df = pd.read_csv('./df_train.csv')
test_df = pd.read_csv('./df_test.csv')
dev_df = pd.read_csv('./df_dev.csv')

In [7]:
display(train_df.head())

display(test_df.head())

display(dev_df.head())

,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Jakarta : Presiden Susilo Bamba...","Menurut Presiden Susilo Bambang Yudhoyono, kem..."
1,1,"Liputan6. com, Jakarta : Perdana Menteri Jepan...",Pada masa silam Jepang terlalu ambisius untuk ...
2,2,"Liputan6. com, Kutai : Banjir dengan ketinggia...",Puluhan hektare areal persawahan yang sebagian...
3,3,"Liputan6. com, Jakarta : Presiden Susilo Bamba...",Sekjen PBB Kofi Annan memuji langkah Presiden ...
4,4,"Liputan6. com, Solok : Warga Kampung Batu Dala...",Untuk mempercepat pelaksanaan belajar-mengajar...


,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Jakarta : Pemerintah masih memb...",Pemerintah memberikan tenggat 14 hari kepada p...
1,1,"Liputan6. com, Jakarta : Diperkirakan 11 juta ...",Satu dari 20 orang Indonesia diperkirakan meng...
2,2,"Liputan6. com, Banda Aceh : Aksi peledakan kan...",Peledakan bom kembali terjadi di Aceh. Kali in...
3,3,"Liputan6. com, Surabaya : Petugas Kepolisian R...",Polres Surabaya Timur menangkap seorang penged...
4,4,"Liputan6. com, Jakarta : Yogyakarta dan Bali m...",Lima seniman Yogyakarta dan Bali menggelar pam...


,Unnamed: 0,clean_article,clean_summary
0,0,"Liputan6. com, Jakarta : Kepolisian Daerah Ria...",Kapolda Riau baru Brigjen Pol. Johny Yodjana b...
1,1,"Liputan6. com, Jakarta : Bank Indonesia dinila...",Kendati Bank Sentral AS menurunkan suku bungan...
2,2,"Liputan6. com, Jakarta : Berbagai kendala meng...",Pemerintah bermaksud akan lebih mengandalkan s...
3,3,"Liputan6. com, Jakarta : Penghapusan beberapa ...","Revisi Kepmennaker Nomor 78 Tahun 2001, dinila..."
4,4,"Liputan6. com, Jakarta : Operasi Sadar Jaya ya...",Polisi menangkap 32 pengunjung Diskotik Mileni...


In [8]:
display(train_df.info())
print('*'*50)
display(test_df.info())
print('*'*50)
display(dev_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193883 entries, 0 to 193882
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     193883 non-null  int64 
 1   clean_article  193883 non-null  object
 2   clean_summary  193883 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.4+ MB


None

**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     10972 non-null  int64 
 1   clean_article  10972 non-null  object
 2   clean_summary  10972 non-null  object
dtypes: int64(1), object(2)
memory usage: 257.3+ KB


None

**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10972 entries, 0 to 10971
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     10972 non-null  int64 
 1   clean_article  10972 non-null  object
 2   clean_summary  10972 non-null  object
dtypes: int64(1), object(2)
memory usage: 257.3+ KB


None

## HuggingFace Data Format

In [9]:
train_data = Dataset.from_dict(train_df)
test_data = Dataset.from_dict(test_df)
val_data = Dataset.from_dict(dev_df) # development data for validation in training

# Build the Model

## Load TinyBERT

In [11]:
bert_model = "cahya/bert2bert-indonesian-summarization" # prajjwal1/bert-tiny #
bert_tokenizer = BertTokenizer.from_pretrained(bert_model)
bert_tokenizer.bos_token =  bert_tokenizer.cls_token
bert_tokenizer.eos_token =  bert_tokenizer.sep_token

## Build the Model

In [12]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

config_encoder = BertConfig(
    vocab_size = len(bert_tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 36,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 768,
)
config_decoder = BertConfig(
    vocab_size = len(bert_tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 36,
    num_hidden_layers = 6,
    num_attention_heads = 6,
    intermediate_size = 768,
)

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = EncoderDecoderModel(config=config)
model.config.decoder_start_token_id = bert_tokenizer.cls_token_id
model.config.pad_token_id = bert_tokenizer.pad_token_id

In [14]:
bert_tokenizer.bos_token, bert_tokenizer.eos_token, bert_tokenizer.get_vocab()[bert_tokenizer.bos_token], bert_tokenizer.get_vocab()[bert_tokenizer.eos_token]

('[CLS]', '[SEP]', 3, 1)

## Get Feaures

In [38]:
def get_feature(batch):
  encodings = bert_tokenizer(batch['clean_article'], text_target=batch['clean_summary'],
                        max_length=512, truncation=True)

  encodings = {'input_ids': encodings['input_ids'],
               'attention_mask': encodings['attention_mask'],
               'labels': encodings['labels']}

  return encodings

In [17]:
prefix = bert_tokenizer.bos_token + " "
posfix = " " + bert_tokenizer.eos_token

def preprocess_function(examples):
    inputs = [prefix + str(doc) + posfix for doc in examples["clean_article"]]
    model_inputs = bert_tokenizer(inputs, max_length=512, truncation=True)
    labels = bert_tokenizer(text=examples["clean_summary"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
data_pt_train = train_data.map(preprocess_function, batched=True)
data_pt_test = test_data.map(preprocess_function, batched=True)
data_pt_val = val_data.map(preprocess_function, batched=True)

Map: 100%|██████████| 10972/10972 [01:34<00:00, 115.80 examples/s]


In [19]:
display(data_pt_train)
display(data_pt_test)
display(data_pt_val)

Dataset({
    features: ['Unnamed: 0', 'clean_article', 'clean_summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 193883
})

Dataset({
    features: ['Unnamed: 0', 'clean_article', 'clean_summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10972
})

Dataset({
    features: ['Unnamed: 0', 'clean_article', 'clean_summary', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10972
})

### Check Data

In [20]:
data_pt_train['input_ids'][0]

[3,
 3,
 17715,
 1050,
 17,
 3036,
 15,
 2647,
 29,
 2601,
 14977,
 9183,
 15137,
 8220,
 1738,
 8511,
 2997,
 1510,
 13339,
 2667,
 16,
 2667,
 3065,
 1509,
 3549,
 1793,
 1542,
 1566,
 3212,
 11365,
 1510,
 2059,
 5576,
 17,
 15137,
 8711,
 3212,
 1542,
 1637,
 16770,
 2814,
 1570,
 5885,
 2855,
 2230,
 3065,
 16,
 3549,
 17,
 3290,
 9537,
 15137,
 1793,
 4478,
 22684,
 3065,
 16,
 3549,
 1495,
 2647,
 17527,
 11399,
 15,
 2647,
 15,
 9002,
 11,
 3082,
 18,
 23,
 12,
 34,
 12287,
 29,
 2601,
 15137,
 2771,
 4478,
 1814,
 1007,
 36,
 17,
 1535,
 2091,
 22699,
 15,
 15137,
 1855,
 4417,
 1572,
 1911,
 13323,
 1507,
 7275,
 28131,
 2242,
 4510,
 6011,
 1509,
 11199,
 1495,
 31929,
 15464,
 4121,
 14140,
 1509,
 14043,
 15,
 3596,
 2207,
 17,
 3475,
 15,
 15137,
 8444,
 4517,
 1542,
 6765,
 10338,
 10214,
 2259,
 15,
 1920,
 4152,
 2066,
 1572,
 9782,
 3212,
 1510,
 13339,
 2076,
 6482,
 17,
 2606,
 6633,
 5885,
 1542,
 1609,
 10061,
 5374,
 3458,
 6276,
 2667,
 16,
 2667,
 3040,
 3713,


In [22]:
data_pt_train['attention_mask'][0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [21]:
data_pt_train['labels'][0]

[3,
 2379,
 2601,
 14977,
 9183,
 15137,
 15,
 11365,
 3244,
 1637,
 3212,
 7695,
 1510,
 13339,
 2667,
 16,
 2667,
 1495,
 3065,
 1509,
 3549,
 17,
 6729,
 3212,
 1542,
 1637,
 14685,
 2814,
 1570,
 22684,
 3065,
 16,
 3549,
 17,
 1]

In [90]:
from transformers import BertConfig, EncoderDecoderConfig, EncoderDecoderModel

config_encoder = BertConfig(
    vocab_size = len(bert_tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 4,
    num_hidden_layers = 2,
    num_attention_heads = 2,
    intermediate_size = 768,
)
config_decoder = BertConfig(
    vocab_size = len(bert_tokenizer.get_vocab()), # sesuaikan dengan vocab tokenizer
    hidden_size = 4,
    num_hidden_layers = 2,
    num_attention_heads = 2,
    intermediate_size = 768,
)

config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder)
model = EncoderDecoderModel(config=config)
model.config.decoder_start_token_id = bert_tokenizer.cls_token_id
model.config.pad_token_id = bert_tokenizer.pad_token_id

In [91]:
import evaluate

rouge = evaluate.load("rouge")
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = bert_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, bert_tokenizer.pad_token_id)
    decoded_labels = bert_tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != bert_tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [92]:
from transformers import DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
data_collator = DataCollatorForSeq2Seq(tokenizer=bert_tokenizer, model='t5-small')

In [93]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    warmup_steps=500,
    logging_steps=100,
    gradient_accumulation_steps=32,
    fp16=False
)

In [94]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=data_pt_train,
    eval_dataset=data_pt_val,
    tokenizer=bert_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [95]:
trainer.train()

  0%|          | 0/6058 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a mod

{'loss': 10.3704, 'learning_rate': 1e-05, 'epoch': 0.02}


  3%|▎         | 200/6058 [04:03<1:35:51,  1.02it/s]

{'loss': 10.3664, 'learning_rate': 2e-05, 'epoch': 0.03}


  5%|▍         | 300/6058 [05:44<1:44:06,  1.08s/it]

...

 99%|█████████▉| 6000/6058 [1:48:47<00:51,  1.14it/s]C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:619: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  decoder_attention_mask = decoder_input_ids.new_tensor(decoder_input_ids != self.config.pad_token_id)
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\models\encoder_decoder\modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine

{'loss': 9.4246, 'learning_rate': 5.217704210147535e-07, 'epoch': 0.99}


100%|██████████| 6058/6058 [1:49:39<00:00,  1.05it/s]























































































































































































































































































































































                                                     

100%|██████████| 6058/6058 [1:52:32<00:00,  1.05it/s]


100%|██████████| 6058/6058 [1:52:32<00:00,  1.11s/it]

{'eval_loss': 9.425400733947754, 'eval_runtime': 172.89, 'eval_samples_per_second': 63.462, 'eval_steps_per_second': 1.984, 'epoch': 1.0}
{'train_runtime': 6752.8123, 'train_samples_per_second': 28.711, 'train_steps_per_second': 0.897, 'train_loss': 9.77579588715335, 'epoch': 1.0}


TrainOutput(global_step=6058, training_loss=9.77579588715335, metrics={'train_runtime': 6752.8123, 'train_samples_per_second': 28.711, 'train_steps_per_second': 0.897, 'train_loss': 9.77579588715335, 'epoch': 1.0})

### Filter Data

In [96]:
article = test_data['clean_article'][0]
summary = test_data['clean_summary'][0]

In [97]:
def predict_summary(document):
  device = model.device
  tokenized = bert_tokenizer([document], max_length=512, truncation =True, padding ='longest',return_tensors='pt')
  tokenized = {k: v.to(device) for k, v in tokenized.items()}
  tokenized_result = model.generate(**tokenized, max_length=128)
  tokenized_result = tokenized_result.to('cpu')
  predicted_summary = bert_tokenizer.decode(tokenized_result[0])
  return predicted_summary

In [98]:
predicted_summary = predict_summary(article)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\generation\utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [99]:
predicted_summary

'[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,'

In [100]:
summary

'Pemerintah memberikan tenggat 14 hari kepada para konglomerat penandatangan MSAA untuk menyerahkan aset. Jika mangkir, mereka bakal dihukum.'

In [104]:
from rouge import Rouge

def get_rouge_scores(actual_summary, predicted_summary):
    rouge = Rouge()
    scores = rouge.get_scores(predicted_summary, actual_summary)
    return [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']]

In [105]:
# rouge score of validation data

from tqdm import tqdm
import pandas as pd

rouge1_scores = []
rouge2_scores = []
rougel_scores = []

pred_summary_list = []

for i in tqdm(range(len(data_pt_test['clean_summary']))):

  doc = data_pt_test['clean_article'][i]
  pred_summary = predict_summary(doc)
  human_summary = data_pt_test['clean_summary'][i]

  score = get_rouge_scores(human_summary, pred_summary)

  rouge1_scores.append(score[0])
  rouge2_scores.append(score[1])
  rougel_scores.append(score[2])

  pred_summary_list.append(pred_summary)

eval = pd.DataFrame({
    "pred_summary" : pred_summary_list,
    "rouge1" : rouge1_scores,
    "rouge2" : rouge2_scores,
    "rougel" : rougel_scores,
})


  0%|          | 0/10972 [00:00<?, ?it/s]

100%|██████████| 10972/10972 [1:14:36<00:00,  2.45it/s]


In [106]:
# average rouge 1
eval['rouge1'].mean()

0.0

In [107]:
# average rouge 2
eval['rouge2'].mean()

0.0

In [108]:
# average rouge l
eval['rougel'].mean()

0.0

In [109]:
eval

,pred_summary,rouge1,rouge2,rougel
0,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
1,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
2,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
3,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
4,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
...,...,...,...,...
10967,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
10968,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
10969,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
10970,"[CLS],,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...",0.0,0.0,0.0
